In [1]:
import pandas as pd, numpy as np
from pathlib import Path

MASTER = "../data/splits/master.csv"
OUTDIR = Path("../data/splits"); OUTDIR.mkdir(parents=True, exist_ok=True)
SEED = 42

df = pd.read_csv(MASTER)
df["race_key"]   = df["race_cat"].fillna("UNK")
df["gender_key"] = df["gender_cat"].fillna("UNK")
df["stratum"] = df["source_dataset"].astype(str)+"|"+df["race_key"].astype(str)+"|"+df["gender_key"].astype(str)

parts = []
for k,g in df.groupby("stratum", dropna=False):
    g = g.sample(frac=1, random_state=SEED)
    n = len(g); n_tr, n_val = int(0.7*n), int(0.15*n)
    splits = (["train"]*n_tr)+(["val"]*n_val)+(["test"]*(n-n_tr-n_val))
    parts.append(g.assign(split=splits)[["image_id","split"]])

splits_df = pd.concat(parts)
for s in ["train","val","test"]:
    splits_df.query("split == @s")[["image_id"]].to_csv(OUTDIR/f"{s}.csv", index=False)
print("Wrote data/splits/train.csv, val.csv, test.csv")


Wrote data/splits/train.csv, val.csv, test.csv
